![alt text](./pageheader_rose2_babies.jpg)

## Analysis and visualisation of ventilator inflations using pressure and flow data obtained with 100 Hz sampling rate and the output of the Ventiliser package.

#### Author: Dr Gusztav Belteki

This notebook imports fast (100 Hz) `pressure` and `flow` data of selected recordings. I also imports the annotation (breath prediction) files generated by _Ventiliser_. It calculates the number of triggered and backup inflations in the recordings and writes it into a text file together with other basic characteristics of the recording.

After this, two 1-hour segments of the ventilation are chosen and analysed quantitatively for details of each ventilator inflation and ther sub-phases. The characteristics of the two periods are compared using statistical methods. Single inflations and multiple conscecutive inflations are visualized as waveforms or loops. These graphs are also exported as image files.

This Notebook also generates Table 4, Figure 4 and Supplementary Figure 5 & 6 as presented in the paper describing _Ventiliser_: David Chong, Colin J Morley & Gusztav Belteki: __Computational analysis of neonatal ventilator waveforms and loop__. _Pediatric Research_, in press. 

The _Ventiliser_ Python package presented in the paper can be downloaded from [here](https://github.com/barrinalo/Ventiliser).

With questions and for more information please contact us: gbelteki@aol.com (Gusztav Belteki, dtwc3@cam.ac.uk (David Chong)

### 1. Import the required libraries and set options

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas import DataFrame
from scipy import stats
from matplotlib import dates

import os
import datetime

# Module containing functions to import and process data
import fast_data_analyser

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

# Suppress chained assigment warning
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
DRIVE_OUT = '/Volumes/guszti'
TOPIC = 'analysis_individual'

# Select the recording to analyse
recording = 'DG007'

### 2. Import ventilator data

This importing scripts takes approximately 20 seconds / Gbyte

In [ ]:
%%time

fast_data = fast_data_analyser.process_fast_data(recording)

### 3. Import annotations

These annotations were generated by _Ventiliser_

In [ ]:
%%time

predicted_breaths = fast_data_analyser.process_predicted_breaths(recording)

### 4. Basic reports on the whole recording

#### Synchronized and backup inflations and spontaneous breaths

During `spontaneous breaths` the the correlation between pressure and flow is negative (during ventilator inflations it is positive). 

During `synchronized inflations` the pressure start rising **>=30 mseconds after** the start of the positive flow generated by the patient (trigger delay).

During `backup inflations` the flow is rising at the same time or after the pressure rise (but these are calculated as not spontaneous breaths and not synchronized inflations)

In [ ]:
spontaneous = {}
backup = {}
synchronized = {}
    
for part in fast_data:
        
    spontaneous[part] = np.sum(predicted_breaths[part]['pressure_flow_correlation'] < 0)
        
    synchronized[part] = np.sum(
        (predicted_breaths[part]['pressure_flow_correlation'] >= 0) &
        (predicted_breaths[part]['pressure_rise_start'] >= 
            predicted_breaths[part]['inspiration_initiation_start'] + 3))
        
    backup[part] = (predicted_breaths[part].shape[0] - (spontaneous[part] + synchronized[part]))

#### How long are the fast data, how many predicted breaths, how many is synchronised and backup?

In [ ]:
print(recording)
for part in fast_data:
    print(part)
    print('Start: %s' % fast_data[part].index[0])
    print('End: %s' % fast_data[part].index[-1])
    print('Length of ventilation data: ', len(fast_data[part]))
    print('Total number of inflations: ', len(predicted_breaths[part]))
    print('Number of synchronized inflation: ', synchronized[part])
    print('Number of backup inflation: ', backup[part])
    print('Number of spontaneous breaths: ', spontaneous[part], '\n')       

In [ ]:
if not os.path.isdir(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording)):
    os.makedirs(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording))
DATA_DUMP = os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording)
    
fle = open(os.path.join(DATA_DUMP, '%s_annotation_data.txt' % recording), 'w')
    
print(recording, '\n', file = fle)
for part in fast_data:
    print(part, '\n', file = fle)
    print('Start: %s' % fast_data[part].index[0], file = fle)
    print('End: %s' % fast_data[part].index[-1], file = fle)
    print('Length of ventilation data: ', len(fast_data[part]), file = fle)
    print('Total number of inflations: ', len(predicted_breaths[part]), file = fle)
    print('Number of synchronised inflations: ', synchronized[part], file = fle)
    print('Number of backup inflations: ', backup[part], file = fle)
    print('Number of spontaneous breaths: ', spontaneous[part], '\n', file = fle)
fle.close()

### 5. Choose a part of the recording (if there is more than one file)

In [ ]:
list(fast_data.keys())

In [ ]:
part = sorted(fast_data.keys())[0]
print(part)

### 6. Descriptive statistics about ventilator phases and subphases in the whole recording

In [ ]:
predicted_breaths[part].head()

In [ ]:
# Only consider ventilator inflations, exclude spontaneous breaths with 
# negative pressure-flow correlations
inflations_all = predicted_breaths[part][predicted_breaths[part]['pressure_flow_correlation'] > 0].copy()

##### Statistics about the length of inspiratory and expiratory hold in those inflations who have it

In [ ]:
# How many inflations have inspiratory hold ?
len(inflations_all[inflations_all['inspiratory_hold_length']>0]), len(inflations_all)

In [ ]:
# What is the proportion of inflations with inspiratory hold ?
len(inflations_all[inflations_all['inspiratory_hold_length']>0]) / len(inflations_all)

In [ ]:
insp_hold_stats_all = \
    inflations_all[inflations_all['inspiratory_hold_length']>0]['inspiratory_hold_length'].describe(percentiles = 
                                            [0.01, 0.1, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
insp_hold_stats_all = DataFrame(insp_hold_stats_all)
insp_hold_stats_all

In [ ]:
# How many inflations have expiratory hold ?
len(inflations_all[inflations_all['expiratory_hold_length']>0]), len(inflations_all)

In [ ]:
# What is the proportion of inflations with expiratory hold ?
len(inflations_all[inflations_all['expiratory_hold_length']>0]) / len(inflations_all)

In [ ]:
exp_hold_stats_all = \
    inflations_all[inflations_all['expiratory_hold_length']>0]['expiratory_hold_length'].describe(percentiles = 
                                            [0.01, 0.1, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
exp_hold_stats_all = DataFrame(exp_hold_stats_all)
exp_hold_stats_all

##### Statistics about the duration of various ventilator subphases and calculated parameters

In [ ]:
parameters_to_keep = ['inspiratory_hold_length', 'pressure_rise_length', 'pip_length', 'pressure_drop_length',
                       'peep_length', 'expiratory_hold_length', 'pip_to_no_flow_length', 'peep_to_no_flow_length',
                       'lung_inflation_length', 'total_inspiratory_length',
                       'lung_deflation_length', 'total_expiratory_length',
                       'inspiratory_volume', 'expiratory_volume', 'max_inspiratory_flow',
                       'max_expiratory_flow', 'max_pressure', 'min_pressure']

subphase_stats_all  = inflations_all[parameters_to_keep].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]).T
subphase_stats_all

In [ ]:
insp_hold_stats_all.to_csv('%s/%s_%s_insp_hold_stats_all.csv' % (DATA_DUMP, recording, part[:-4]))
exp_hold_stats_all.to_csv('%s/%s_%s_exp_hold_stats_all.csv' % (DATA_DUMP, recording, part[:-4]))
subphase_stats_all.to_csv('%s/%s_%s_subphase_stats_all.csv' % (DATA_DUMP, recording, part[:-4]))

# `PERIOD 1`

### 7. Choose a period of the this recording to study further

Processing time will be approximately 1 minute / 1000 breaths. As approximatelly 1 breath happens every second, this corresponds to ~15 minutes of recording. Therefore, it is not really feasible to choose a period longer than 1 hour unless you are prepared for this step to take more than ~5 minutes. With longer periods, processing time will increase linearly.

Choose the start and end time in `2019-03-20 12:34:18` format

In [ ]:
fast_data[part].head()

In [ ]:
fast_data[part].tail()

In [ ]:
start_time = '2015-10-19 22:00:00'
end_time =   '2015-10-19 23:00:00'

dir_tag = '%s%s%s_%s%s%s_%s%s%s' % (start_time[:4], start_time[5:7], start_time[8:10], 
                                start_time[11:13], start_time[14:16], start_time[17:19],
                                 end_time[11:13], end_time[14:16], end_time[17:19])

print(dir_tag)

In [ ]:
# Images and raw data will be written on an external hard drive
if not os.path.isdir(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag)):
    os.makedirs(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag))
DATA_DUMP_1 = os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag)
DATA_DUMP_1

In [ ]:
start_time = pd.Timestamp(start_time)
end_time =   pd.Timestamp(end_time)

start_data_point = start_time - fast_data[part].index[0]
end_data_point = end_time - fast_data[part].index[0]
end_data_point - start_data_point 

In [ ]:
start_time, end_time

In [ ]:
start_data_point, end_data_point

In [ ]:
a = predicted_breaths[part]
                                 
period_1 = a[(a['breath_start'] > int(start_data_point.total_seconds() * 100)) & 
           (a['breath_start'] < int(end_data_point.total_seconds() * 100) )]

print('Number of breaths: %s' % len(period_1))

In [ ]:
period_1.head()

In [ ]:
period_1.tail()

In [ ]:
start = period_1.index[0]
end = period_1.index[-1]

br_selected = list(range(start, end+1))
tag = '%d_%d' % (start, end)

In [ ]:
period_1.to_csv('%s/%s_%s_period%s_breaths_all_annotation.csv' % (DATA_DUMP_1, recording, part[:-4], tag))

In [ ]:
print(recording, part[:-17], tag)

In [ ]:
breaths = {}
for br in br_selected:
    breaths[br] = fast_data[part][predicted_breaths[part].iloc[br,1] : predicted_breaths[part].iloc[br,2]]

The next code will take  `approximately 1 minute / 1500 breaths`

In [ ]:
%%time
breaths_1 = fast_data_analyser.flow_integrater(breaths)

In [ ]:
breaths_all_1 = pd.concat(breaths_1)

In [ ]:
breaths_all_1.head()

In [ ]:
breaths_all_1.tail()

In [ ]:
breaths_all_1.to_csv('%s/%s_%s_period%s_breaths_all.csv' % (DATA_DUMP_1, recording, part[:-4], tag))

In [ ]:
print('Number of data points: %d' % len(breaths_all_1))

In [ ]:
# number of inflations
breaths_all_1.index[-1][0] - breaths_all_1.index[0][0]

In [ ]:
synchronized_1 = {}
backup_1 = {}
spontaneous_1 = {}

for br in breaths_1:
    
    if predicted_breaths[part]['pressure_flow_correlation'].iloc[br] < 0:
        spontaneous_1[br] = breaths_1[br]
    elif predicted_breaths[part]['pressure_rise_start'].iloc[br] >= \
            predicted_breaths[part]['inspiration_initiation_start'].iloc[br] + 3:
        synchronized_1[br] = breaths_1[br]
    else:
        backup_1[br] = breaths_1[br]


synchronized_all_1 = pd.concat(synchronized_1)
backup_all_1 = pd.concat(backup_1)
#spontaneous_all_1 = pd.concat(spontaneous_1)

In [ ]:
len(synchronized_1), len(backup_1)

In [ ]:
synchronized_all_1.to_csv('%s/%s_%s_period%s_synchronised_all.csv' % (DATA_DUMP_1, recording, part[:-4], tag))
backup_all_1.to_csv('%s/%s_%s_period%s_backup_all.csv' % (DATA_DUMP_1, recording, part[:-4], tag))
#spontaneous_all_1.to_csv('%s/%s_%s_period%s_spontaneous_all.csv' % (DATA_DUMP_1, recording, part[:-4], tag))

### 8. Basic report on the selected period

In [ ]:
print('Recording:', recording,)
print('Part:', '%s' % part,)
print('Period:', '%s' % tag, '\n')
print('Start time %s' % start_time)
print('End time %s' % end_time) 
print('Number of data points %d' % len(breaths_all_1)) 
print('Number of inflations %d' %  len(period_1))
print('Number of synchronised inflations: %d' % len(synchronized_all_1.index.levels[0]))
print('Number of backup inflations: %d' % len(backup_all_1.index.levels[0]))
#print('Number of spontaneous breaths: %d' % len(spontaneous_all_1.index.levels[0]), '\n')

In [ ]:
# Write report to text file

fle = open('%s/%s_%s_period%s_annotation_data.txt' % (DATA_DUMP_1, recording, part[:-4], tag), 'w')
print('Recording:', recording, file = fle)
print('Part:', '%s' % part, file = fle)
print('Period:', '%s' % tag, '\n',  file = fle)
print('Start time %s' % start_time, file = fle)
print('End time %s' % end_time, file = fle) 
print('Number of data points %d' % len(breaths_all_1), file = fle) 
print('Number of inflations %d' %  len(period_1), file = fle)
print('Number of synchronised inflations: %d' % len(synchronized_all_1.index.levels[0]), file = fle)
print('Number of backup inflations: %d' % len(backup_all_1.index.levels[0]), file = fle)
#print('Number of spontaneous breaths: %d' % len(spontaneous_all_1.index.levels[0]), '\n', file = fle)

fle.close()

### 9. Descriptive statistics about ventilator phases and subphases during the period

In [ ]:
period_1.head()

##### Descriptive statistics about the length of inspiratory hold in those inflations who have it

In [ ]:
# How many inflations have inspiratory hold ?
len(period_1[period_1['inspiratory_hold_length']>0]), len(period_1)

In [ ]:
# What is the proportion of inflations with inspiratory hold ?
len(period_1[period_1['inspiratory_hold_length']>0]) / len(period_1)

In [ ]:
insp_hold_stats_all_1 = \
    period_1[period_1['inspiratory_hold_length']>0]['inspiratory_hold_length'].describe(percentiles = 
        [0.01, 0.1, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
insp_hold_stats_all_1 = DataFrame(insp_hold_stats_all_1)
insp_hold_stats_all_1

In [ ]:
# How many inflations have expiratory hold ?
len(period_1[period_1['expiratory_hold_length']>0]), len(period_1)

In [ ]:
# What is the proportion of inflations with inspiratory hold ?
len(period_1[period_1['expiratory_hold_length']>0]) / len(period_1)

In [ ]:
exp_hold_stats_all_1 = \
    period_1[period_1['expiratory_hold_length']>0]['expiratory_hold_length'].describe(percentiles = 
        [0.01, 0.1, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
exp_hold_stats_all_1 = DataFrame(exp_hold_stats_all_1)
exp_hold_stats_all_1

##### Descriptive statistics about the duration of various ventilator subphases and calculated parameters

In [ ]:
parameters_to_keep = ['inspiratory_hold_length', 'pressure_rise_length', 'pip_length', 'pressure_drop_length',
                       'peep_length', 'expiratory_hold_length', 'pip_to_no_flow_length', 'peep_to_no_flow_length',
                       'lung_inflation_length', 'total_inspiratory_length',
                       'lung_deflation_length', 'total_expiratory_length',
                       'inspiratory_volume', 'expiratory_volume', 'max_inspiratory_flow',
                       'max_expiratory_flow', 'max_pressure', 'min_pressure']

subphase_stats_all_1  = period_1[parameters_to_keep].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]).T
subphase_stats_all_1

In [ ]:
insp_hold_stats_all_1.to_csv('%s/%s_%s_period%s_insp_hold_stats.csv' % (DATA_DUMP_1, recording, part[:-4], tag))
exp_hold_stats_all_1.to_csv('%s/%s_%s_period%s_exp_hold_stats.csv' % (DATA_DUMP_1, recording, part[:-4], tag))
subphase_stats_all_1.to_csv('%s/%s_%s_period%s_subphase_stats.csv' % (DATA_DUMP_1, recording, part[:-4], tag))

### 10. Composite loops of all the breaths of the period

#### Pressure - volume loops

In [ ]:
fast_data_analyser.PV_loop_all(recording, part, tag, breaths_all_1['pressure'], breaths_all_1['volume'], 
    pressure_high=30, vol_high=25, alpha = 0.01, filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_1)

#### Flow-volume loops

In [ ]:
fast_data_analyser.FV_loop_all(recording, part, tag, breaths_all_1['volume'], breaths_all_1['flow'], 
           vol_high = 25, flow_low = -6, flow_high = 8,
           alpha = 0.02, filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_1)

#### Plot synchronized,  backup  and spontaneous inflations separately

In [ ]:
fast_data_analyser.PV_loops_synchr_backup(recording, part, tag, 
                       synchronized_all_1['pressure'], synchronized_all_1['volume'],
                       backup_all_1['pressure'], backup_all_1['volume'],
                       pressure_high = 30, vol_high = 25,  alpha_sync = 0.01, alpha_backup = 0.01,
                       filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_1)

In [ ]:
fast_data_analyser.FV_loops_synchr_backup(recording, part, tag, synchronized_all_1['volume'], synchronized_all_1['flow'],
                       backup_all_1['volume'], backup_all_1['flow'],
                       vol_high = 25, flow_low = -6, flow_high = 8, alpha_sync = 0.01, 
                       alpha_backup = 0.01, filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_1)

### 11. Study and visualize an individual inflation

In [ ]:
# Images and raw data will be written on an external hard drive
if not os.path.isdir(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag, 'inflations')):
    os.makedirs(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag, 'inflations'))
DATA_DUMP_2 = os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag, 'inflations')
DATA_DUMP_2

In [ ]:
print(recording, part, tag)

In [ ]:
br = 2800
br

In [ ]:
breaths[br].head()

In [ ]:
print(predicted_breaths[part].iloc[br].apply(lambda x: round(x, 2)))

In [ ]:
# Write report to text file

fle = open('%s/%s_%s_breath%d_annotation_data.txt' % (DATA_DUMP_2, recording, part[:-4], br), 'w')
print(recording, file = fle)
print('Part %s' % part, '\n', file = fle)
print('Breath %d' % br, '\n', file = fle)
print(predicted_breaths[part].iloc[br].apply(lambda x: round(x, 2)), file = fle)
fle.close()

#### Visuzalize individual waves and loops

In [ ]:
fast_data_analyser.wave_individual(recording, part, breaths[br], br, write = True, folder = DATA_DUMP_2)

In [ ]:
fast_data_analyser.loops(recording, part, breaths[br], br, write = True, folder = DATA_DUMP_2)

### 12. Choose randomly a number of inflations and generate reports and graphs on them

In [ ]:
for i in range(5): # 5 random inflations
    br = np.random.randint(start, end)
    
    # Write report to text file
    fle = open('%s/%s_%s_breath%d_annotation_data.txt' % (DATA_DUMP_2, recording, part[:-4], br), 'w')
    print(recording, file = fle)
    print('Part %s' % part, '\n', file = fle)
    print('Breath %d' % br, '\n', file = fle)
    print(predicted_breaths[part].iloc[br].apply(lambda x: round(x, 2)), file = fle)
    fle.close()
    
    # Export images with waveforms and loops for individual breaths
    fast_data_analyser.wave_individual(recording, part, breaths[br], br, show = False, 
                                       write = True, folder = DATA_DUMP_2)
    fast_data_analyser.loops(recording, part, breaths[br], br, show = False, 
                                       write = True, folder = DATA_DUMP_2)

### 13. Visualize multiple inflations as waveforms and loops

In [ ]:
print(recording, part, tag)

In [ ]:
start_seq = start; end_seq = start_seq + 8 
brs_range = np.arange(start_seq, end_seq+1)

In [ ]:
breaths_selected = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected = pd.concat(breaths_selected, sort = True)
breaths_selected = breaths_selected.reset_index(level = 0)

In [ ]:
len(breaths_selected)

In [ ]:
breaths_selected.head()

In [ ]:
breaths_selected.tail()

In [ ]:
fast_data_analyser.waves_multiple(recording, part, breaths_selected, write = True, folder = DATA_DUMP_2)

In [ ]:
fast_data_analyser.loops_multiple(recording, part, breaths_selected, write = True, folder = DATA_DUMP_2)

### 14. Choose randomly a number of inflations sequence and generate graphs on them

In [ ]:
def write_multiple_waves_loops(samples = 3, number_of_breaths = 8, show = True):
    
    # Default: 3 samples of 8 inflations each
    for i in range(samples):
        first_breath = np.random.randint(start, end)
        brs = np.arange(first_breath, first_breath + number_of_breaths)
        if show:
            print(brs)
        breaths_selected = {key: value for key, value in breaths.items() if key in brs}
        breaths_selected = pd.concat(breaths_selected, sort = True)
        breaths_selected = breaths_selected.reset_index(level = 0)
        
        fast_data_analyser.waves_multiple(recording, part, breaths_selected, show = False, 
                                          write = True,  folder = DATA_DUMP_2)
        fast_data_analyser.loops_multiple(recording, part,breaths_selected, show = False, 
                                          write = True,  folder = DATA_DUMP_2)

In [ ]:
# 5 samples, with 8 breaths in each
write_multiple_waves_loops(samples = 5, number_of_breaths = 8) 

In [ ]:
# 3 samples, with 40 breaths in each
write_multiple_waves_loops(samples = 3, number_of_breaths = 40) 

In [ ]:
# 1 sample with 200 breaths
write_multiple_waves_loops(samples = 1, number_of_breaths = 200, show = False) 

# `PERIOD 2`

### 15. Choose another period of the this recording to study further

Processing time will be approximately 1 minute / 1000 breaths. As approximatelly 1 breath happens every second, this corresponds to ~15 minutes of recording. Therefore, it is not really feasible to choose a period longer than 1 hour unless you are prepared for this step to take more than ~5 minutes. With longer periods, processing time will increase linearly.

Choose the start and end time in `2019-03-20 12:34:18` format

In [ ]:
fast_data[part].head()

In [ ]:
fast_data[part].tail()

In [ ]:
start_time = '2015-10-20 17:00:00'
end_time =   '2015-10-20 18:00:00'

dir_tag = '%s%s%s_%s%s%s_%s%s%s' % (start_time[:4], start_time[5:7], start_time[8:10], 
                                start_time[11:13], start_time[14:16], start_time[17:19],
                                 end_time[11:13], end_time[14:16], end_time[17:19])

print(dir_tag)

In [ ]:
# Images and raw data will be written on an external hard drive
if not os.path.isdir(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag)):
    os.makedirs(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag))
DATA_DUMP_3 = os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag)
DATA_DUMP_3

In [ ]:
start_time = pd.Timestamp(start_time)
end_time =   pd.Timestamp(end_time)

start_data_point = start_time - fast_data[part].index[0]
end_data_point = end_time - fast_data[part].index[0]
end_data_point - start_data_point 

In [ ]:
a = predicted_breaths[part]
                                 
period_2 = a[(a['breath_start'] > int(start_data_point.total_seconds() * 100)) & 
           (a['breath_start'] < int(end_data_point.total_seconds() * 100) )]

print('Number of inflations: %s' % len(period_2))

In [ ]:
period_2.head()

In [ ]:
period_2.tail()

In [ ]:
start = period_2.index[0]
end = period_2.index[-1]

br_selected = list(range(start, end+1))
tag = '%d_%d' % (start, end)

In [ ]:
period_2.to_csv('%s/%s_%s_period%s_breaths_all_annotation.csv' % (DATA_DUMP_3, recording, part[:-4], tag))

In [ ]:
print(recording, part[:-4], tag)

In [ ]:
breaths = {}
for br in br_selected:
    breaths[br] = fast_data[part][predicted_breaths[part].iloc[br,1] : 
                                             predicted_breaths[part].iloc[br,2]]

The next code will take  `approximately 1 minute / 1500 breaths`

In [ ]:
%%time
breaths_2 = fast_data_analyser.flow_integrater(breaths)

In [ ]:
breaths_all_2 = pd.concat(breaths_2)

In [ ]:
breaths_all_2.head()

In [ ]:
breaths_all_2.tail()

In [ ]:
breaths_all_2.to_csv('%s/%s_%s_period%s_breaths_all.csv' % (DATA_DUMP_3, recording, part[:-4], tag))

In [ ]:
print('Number of data points: %d' % len(breaths_all_2))

In [ ]:
# number of inflations
breaths_all_2.index[-1][0] - breaths_all_2.index[0][0]

In [ ]:
synchronized_2 = {}
backup_2 = {}
spontaneous_2 = {}

for br in breaths:
    
    if predicted_breaths[part]['pressure_flow_correlation'].iloc[br] < 0:
        spontaneous_2[br] = breaths[br]
    elif predicted_breaths[part]['pressure_rise_start'].iloc[br] >= \
            predicted_breaths[part]['inspiration_initiation_start'].iloc[br] + 3:
        synchronized_2[br] = breaths[br]
    else:
        backup_2[br] = breaths[br]


synchronized_all_2 = pd.concat(synchronized_2)
backup_all_2 = pd.concat(backup_2)
spontaneous_all_2 = pd.concat(spontaneous_2)

In [ ]:
synchronized_all_2.to_csv('%s/%s_%s_period%s_synchronised_all.csv' % (DATA_DUMP_3, recording, part[:-4], tag))
backup_all_2.to_csv('%s/%s_%s_period%s_backup_all.csv' % (DATA_DUMP_3, recording, part[:-4], tag))
spontaneous_all_2.to_csv('%s/%s_%s_period%s_spontaneous_all.csv' % (DATA_DUMP_3, recording, part[:-4], tag))

### 16. Basic report on the selected period

In [ ]:
print('Recording:', recording,)
print('Part:', '%s' % part,)
print('Period:', '%s' % tag, '\n')
print('Start time %s' % start_time)
print('End time %s' % end_time) 
print('Number of data points %d' % len(breaths_all_2)) 
print('Number of inflations %d' %  len(period_2))
print('Number of synchronised inflations: %d' % len(synchronized_all_2.index.levels[0]))
print('Number of backup inflations: %d' % len(backup_all_2.index.levels[0]))
print('Number of spontaneous breaths: %d' % len(spontaneous_all_2.index.levels[0]), '\n')

In [ ]:
# Write report to text file

fle = open('%s/%s_%s_period%s_annotation_data.txt' % (DATA_DUMP_3, recording, part[:-4], tag), 'w')
print('Recording:', recording, file = fle)
print('Part:', '%s' % part, file = fle)
print('Period:', '%s' % tag, '\n',  file = fle)
print('Start time %s' % start_time, file = fle)
print('End time %s' % end_time, file = fle) 
print('Number of data points %d' % len(breaths_all_2), file = fle) 
print('Number of inflations %d' %  len(period_2), file = fle)
print('Number of synchronised inflations: %d' % len(synchronized_all_2.index.levels[0]), file = fle)
print('Number of backup inflations: %d' % len(backup_all_2.index.levels[0]), file = fle)
print('Number of spontaneous breaths: %d' % len(spontaneous_all_2.index.levels[0]), '\n', file = fle)

fle.close()

### 17. Descriptive statistics about ventilator phases and subphases during the period

In [ ]:
period_2.head()

In [ ]:
# For analysis of sub-phase duration consider only ventilator inflations

period_2_vent = period_2[period_2['pressure_flow_correlation'] > 0]

In [ ]:
len(period_2), len(period_2_vent)

##### Statistics about the length of inspiratory and expiratory hold in those inflations who have it

In [ ]:
# How many inflations have inspiratory hold ?
len(period_2_vent[period_2_vent['inspiratory_hold_length']>0]), len(period_2_vent)

In [ ]:
# What is the proportion of inflations with inspiratory hold ?
len(period_2_vent[period_2_vent['inspiratory_hold_length']>0]) / len(period_2_vent)

In [ ]:
insp_hold_stats_all_2 = \
    period_2_vent[period_2_vent['inspiratory_hold_length']>0]['inspiratory_hold_length'].describe(percentiles = 
        [0.01, 0.1, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
insp_hold_stats_all_2 = DataFrame(insp_hold_stats_all_2)
insp_hold_stats_all_2

In [ ]:
# How many inflations have expiratory hold ?
len(period_2_vent[period_2_vent['expiratory_hold_length']>0]), len(period_2_vent)

In [ ]:
# What is the proportion of inflations with expiratory hold ?
len(period_2_vent[period_2_vent['expiratory_hold_length']>0]) / len(period_2_vent)

In [ ]:
exp_hold_stats_all_2 = \
    period_2_vent[period_2_vent['expiratory_hold_length']>0]['expiratory_hold_length'].describe(percentiles = 
        [0.01, 0.1, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
exp_hold_stats_all_2 = DataFrame(exp_hold_stats_all_2)
exp_hold_stats_all_2

##### Statistics about the duration of various ventilator subphases and calculated parameters

In [ ]:
period_2_vent

In [ ]:
parameters_to_keep = ['inspiratory_hold_length', 'pressure_rise_length', 'pip_length', 'pressure_drop_length',
                       'peep_length', 'expiratory_hold_length', 'pip_to_no_flow_length', 'peep_to_no_flow_length',
                       'lung_inflation_length', 'total_inspiratory_length',
                       'lung_deflation_length', 'total_expiratory_length',
                       'inspiratory_volume', 'expiratory_volume', 'max_inspiratory_flow',
                       'max_expiratory_flow', 'max_pressure', 'min_pressure']

subphase_stats_all_2  = period_2_vent[parameters_to_keep].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]).T
subphase_stats_all_2

In [ ]:
insp_hold_stats_all_2.to_csv('%s/%s_%s_insp_hold_stats_period_2.csv' % (DATA_DUMP_3, recording, part[:-4]))
exp_hold_stats_all_2.to_csv('%s/%s_%s_exp_hold_stats_period_2.csv' % (DATA_DUMP_3, recording, part[:-4]))
subphase_stats_all_2.to_csv('%s/%s_%s_subphase_stats_period_2.csv' % (DATA_DUMP_3, recording, part[:-4]))

### 18. Composite loops of all the breaths of the period

#### Pressure - volume loops

In [ ]:
fast_data_analyser. PV_loop_all(recording, part, tag, breaths_all_2['pressure'], breaths_all_2['volume'], 
    pressure_high=30, vol_high=25, alpha = 0.01, filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_3)

#### Flow-volume loops

In [ ]:
fast_data_analyser.FV_loop_all(recording, part, tag, breaths_all_2['volume'], breaths_all_2['flow'], 
           vol_high = 25, flow_low = -6, flow_high = 8,
           alpha = 0.02, filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_3)

#### Plot synchronized and backup inflations and spontaneous breaths separately

In [ ]:
fast_data_analyser.PV_loops_synchr_backup(recording, part, tag, 
                                          synchronized_all_2['pressure'], synchronized_all_2['volume'],
                       backup_all_2['pressure'], backup_all_2['volume'],
                       pressure_high = 30, vol_high = 25,  alpha_sync = 0.01, alpha_backup = 0.01,
                       filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_3)

In [ ]:
fast_data_analyser.PV_loops_synchr_backup_spont(recording, part, tag, 
                            synchronized_all_2['pressure'], synchronized_all_2['volume'],
                            backup_all_2['pressure'], backup_all_2['volume'],
                            spontaneous_all_2['pressure'], spontaneous_all_2['volume'],
                            pressure_high = 30, vol_high = 25,  alpha_sync = 0.01, alpha_backup = 0.01,
                            alpha_spont = 0.01,
                            filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_3)

In [ ]:
fast_data_analyser.FV_loops_synchr_backup(recording, part, tag, 
                       synchronized_all_2['volume'], synchronized_all_2['flow'],
                       backup_all_2['volume'], backup_all_2['flow'],
                       vol_high = 25, flow_low = -6, flow_high = 8,
                       alpha_sync = 0.01, alpha_backup = 0.01, filetype = 'jpg', dpi = 200, 
                       write = True, folder = DATA_DUMP_3)

In [ ]:
fast_data_analyser.FV_loops_synchr_backup_spont(recording, part, tag, 
                       synchronized_all_2['volume'], synchronized_all_2['flow'],
                       backup_all_2['volume'], backup_all_2['flow'],
                       spontaneous_all_2['volume'], spontaneous_all_2['flow'],      
                       vol_high = 25, flow_low = -6, flow_high = 8,
                       alpha_sync = 0.01, alpha_backup = 0.01, alpha_spont = 0.01,
                       filetype = 'jpg', dpi = 200, write = True, folder = DATA_DUMP_3)

### 19. Study and visualize an individual inflation

In [ ]:
# Images and raw data will be written on an external hard drive
if not os.path.isdir(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag, 'inflations')):
    os.makedirs(os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag, 'inflations'))
DATA_DUMP_4 = os.path.join(DRIVE_OUT, 'data_dump', 'draeger', TOPIC, recording, dir_tag, 'inflations')
DATA_DUMP_4

In [ ]:
print(recording, part, tag)

In [ ]:
br = 73229
br

In [ ]:
breaths[br].head()

In [ ]:
print(predicted_breaths[part].iloc[br].apply(lambda x: round(x, 2)))

In [ ]:
# Write report to text file

fle = open('%s/%s_%s_breath%d_annotation_data.txt' % (DATA_DUMP_4, recording, part[:-4], br), 'w')
print(recording, file = fle)
print('Part %s' % part, '\n', file = fle)
print('Breath %d' % br, '\n', file = fle)
print(predicted_breaths[part].iloc[br].apply(lambda x: round(x, 2)), file = fle)
fle.close()

#### Visuzalize individual waves and loops

In [ ]:
fast_data_analyser.wave_individual(recording, part, breaths[br], br, write = True, folder = DATA_DUMP_4)

In [ ]:
fast_data_analyser.loops(recording, part, breaths[br], br, write = True, folder = DATA_DUMP_4)

### 20. Choose randomly a number of inflations and generate reports and graphs on them

In [ ]:
for i in range(5): # 5 random inflations
    br = np.random.randint(start, end)
    
    # Write report to text file
    fle = open('%s/%s_%s_breath%d_annotation_data.txt' % (DATA_DUMP_4, recording, part[:-4], br), 'w')
    print(recording, file = fle)
    print('Part %s' % part, '\n', file = fle)
    print('Breath %d' % br, '\n', file = fle)
    print(predicted_breaths[part].iloc[br].apply(lambda x: round(x, 2)), file = fle)
    fle.close()
    
    # Export images with waveforms and loops for individual breaths
    fast_data_analyser.wave_individual(recording, part, breaths[br], br, show = False, 
        write = True, folder = DATA_DUMP_4)
    fast_data_analyser.loops(recording, part, breaths[br], br, show = False, 
        write = True, folder = DATA_DUMP_4)

### 21. Visualize multiple inflations as waveforms and loops

In [ ]:
print(recording, part, tag)

In [ ]:
start_seq = start; end_seq = start_seq + 20 
brs_range = np.arange(start_seq, end_seq+1)

In [ ]:
breaths_selected = {key: value for key, value in breaths.items() if key in brs_range}
breaths_selected = pd.concat(breaths_selected, sort = True)
breaths_selected = breaths_selected.reset_index(level = 0)

In [ ]:
len(breaths_selected)

In [ ]:
breaths_selected.head()

In [ ]:
breaths_selected.tail()

In [ ]:
fast_data_analyser.waves_multiple(recording, part, breaths_selected, write = True, folder = DATA_DUMP_3)

In [ ]:
fast_data_analyser.loops_multiple(recording, part, breaths_selected, write = True, folder = DATA_DUMP_3)

### 22. Choose randomly a number of inflations sequence and generate graphs on them

In [ ]:
def write_multiple_waves_loops(samples = 3, number_of_breaths = 8, show = True):
    # Default: 3 samples of 8 inflations each
    for i in range(samples):
        first_breath = np.random.randint(start, end)
        brs = np.arange(first_breath, first_breath + number_of_breaths)
        if show:
            print(brs)
        breaths_selected = {key: value for key, value in breaths.items() if key in brs}
        breaths_selected = pd.concat(breaths_selected, sort = True)
        breaths_selected = breaths_selected.reset_index(level = 0)
        
        fast_data_analyser.waves_multiple(recording, part, breaths_selected, show = False, write = True, folder = DATA_DUMP_3)
        fast_data_analyser.loops_multiple(recording, part, breaths_selected, show = False, write = True, folder = DATA_DUMP_3)

In [ ]:
# 5 samples, with 8 breaths in each
write_multiple_waves_loops(samples = 5, number_of_breaths = 8) 

In [ ]:
# 3 samples, with 40 breaths in each
write_multiple_waves_loops(samples = 3, number_of_breaths = 40) 

In [ ]:
# 1 sample, with 200 breaths in each
write_multiple_waves_loops(samples = 3, number_of_breaths = 200, show = False) 

## 23. Compare period 1 and period 2 with inferential statistics

In [ ]:
period_1.head()

In [ ]:
period_2.head()

In [ ]:
stats.mannwhitneyu(period_1['inspiratory_hold_length'], period_2['inspiratory_hold_length'])

In [ ]:
# Mann-Whitney U tests with Bonferroni correction
MW_tests = {}

pars = ['inspiratory_hold_length', 'expiratory_hold_length',
       'pressure_rise_length', 'pip_length', 'pressure_drop_length', 'peep_length', 
       'lung_inflation_length', 'total_inspiratory_length', 'lung_deflation_length', 'total_expiratory_length']

for par in pars:
    MW_tests[par] = stats.mannwhitneyu(period_1[par], period_2[par])

MW_tests = DataFrame(MW_tests, index = ['statistic', 'p value']).T
# Bonferroni
MW_tests['p value'] = round(MW_tests['p value'] * len(MW_tests), 4)

In [ ]:
MW_tests

##### Inspiratory hold

In [ ]:
# How many inflations have inspiratory hold ?
actual_period_1  = len(period_1[period_1['inspiratory_hold_length']>0])
actual_period_1 , len(period_1)

In [ ]:
# What is the proportion of inflations with inspiratory hold ?
actual_period_1 / len(period_1)

In [ ]:
# How many inflations have inspiratory hold ?
actual_period_2  = len(period_2[period_2['inspiratory_hold_length']>0])
actual_period_2 , len(period_2)

In [ ]:
# What is the proportion of inflations with inspiratory hold ?
actual_period_2 / len(period_2)

In [ ]:
expected_proportion = (actual_period_1 + actual_period_2) / (len(period_1) + len(period_2))
expected_proportion

In [ ]:
expected = expected_proportion * len(period_1), expected_proportion * len(period_2)
expected

In [ ]:
chsqtests_insp_hold = stats.chisquare(f_obs= (actual_period_1, actual_period_2),   # Array of observed counts
                                      f_exp= expected)   # Array of expected counts
chsqtests_insp_hold

##### Expiratory hold

In [ ]:
# How many inflations have expiratory hold ?
actual_period_1  = len(period_1[period_1['expiratory_hold_length']>0])
actual_period_1 , len(period_1)

In [ ]:
# What is the proportion of inflations with expiratory hold ?
actual_period_1 / len(period_1)

In [ ]:
# How many inflations have expiratory hold ?
actual_period_2  = len(period_2[period_2['expiratory_hold_length']>0])
actual_period_2 , len(period_2)

In [ ]:
# What is the proportion of inflations with expiratory hold ?
actual_period_2 / len(period_2)

In [ ]:
expected_proportion = (actual_period_1 + actual_period_2) / (len(period_1) + len(period_2))
expected_proportion

In [ ]:
expected = expected_proportion * len(period_1), expected_proportion * len(period_2)
expected

In [ ]:
chsqtests_exp_hold = stats.chisquare(f_obs= (actual_period_1, actual_period_2),   # Array of observed counts
                                      f_exp= expected)   # Array of expected counts
chsqtests_exp_hold

## 24. Figures for the Paper

##### Figure 4

In [ ]:
dpi = 600
filetype = 'jpg'

fig, ax = plt.subplots(2, 2, figsize = [12, 12])
fig.subplots_adjust(left=0.1, bottom=None, right=0.9, top=None, hspace=0.3, wspace=0.2)

# Figure 4A-B

pressure_high = breaths_all_1['pressure'].max() * 1.1
vol_high = breaths_all_1['volume'].max() * 0.9
    
ax[0,0].scatter(synchronized_all_1['pressure'], synchronized_all_1['volume'], color = 'red', s = 5, alpha = 0.01 )
ax[0,1].scatter(backup_all_1['pressure'], backup_all_1['volume'], color = 'blue', s = 5, alpha = 0.01 )
ax[0,0].set_xlim(0, 30), ax[0,1].set_xlim(0, 30)
ax[0,0].set_ylim(0, 25), ax[0,1].set_ylim(0, 25)
ax[0,0].set_xlabel('Pressure (mbar)'), ax[0,1].set_xlabel('Pressure (mbar)')
ax[0,0].set_ylabel('Volume (mL)'), ax[0,1].set_ylabel('Volume (mL)')
ax[0,0].set_title('Synchronised inflations'), ax[0,1].set_title('Backup inflations')
ax[0,0].grid(True), ax[0,1].grid(True)

# Figure 4C-D

pressure_high = breaths_all_1['pressure'].max() * 1.1
vol_high = breaths_all_2['volume'].max() * 0.9
    
ax[1,0].scatter(synchronized_all_2['pressure'], synchronized_all_2['volume'], color = 'red', s = 5, alpha = 0.01 )
ax[1,1].scatter(backup_all_2['pressure'], backup_all_2['volume'], color = 'blue', s = 5, alpha = 0.01 )
ax[1,0].set_xlim(0, 30), ax[1,1].set_xlim(0, 30)
ax[1,0].set_ylim(0, 25), ax[1,1].set_ylim(0, 25)
ax[1,0].set_xlabel('Pressure (mbar)'), ax[1,1].set_xlabel('Pressure (mbar)')
ax[1,0].set_ylabel('Volume (mL)'), ax[1,1].set_ylabel('Volume (mL)')
ax[1,0].set_title('Synchronised inflations'), ax[1,1].set_title('Backup inflations')
ax[1,0].grid(True), ax[1,1].grid(True)

fig.text(0.04, 0.92, 'A', fontsize = 16) 
fig.text(0.49, 0.92, 'B', fontsize = 16)
fig.text(0.04, 0.48, 'C', fontsize = 16)
fig.text(0.49, 0.48, 'D', fontsize = 16)

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Figure_4_col', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);

In [ ]:
dpi = 600
filetype = 'jpg'

fig, ax = plt.subplots(2, 2, figsize = [12, 12])
fig.subplots_adjust(left=0.1, bottom=None, right=0.9, top=None, hspace=0.3, wspace=0.2)

# Figure 4A-B

pressure_high = breaths_all_1['pressure'].max() * 1.1
vol_high = breaths_all_1['volume'].max() * 0.9
    
ax[0,0].scatter(synchronized_all_1['pressure'], synchronized_all_1['volume'], color = 'black', s = 5, alpha = 0.01 )
ax[0,1].scatter(backup_all_1['pressure'], backup_all_1['volume'], color = 'black', s = 5, alpha = 0.01 )
ax[0,0].set_xlim(0, 30), ax[0,1].set_xlim(0, 30)
ax[0,0].set_ylim(0, 25), ax[0,1].set_ylim(0, 25)
ax[0,0].set_xlabel('Pressure (mbar)'), ax[0,1].set_xlabel('Pressure (mbar)')
ax[0,0].set_ylabel('Volume (mL)'), ax[0,1].set_ylabel('Volume (mL)')
ax[0,0].set_title('Synchronised inflations'), ax[0,1].set_title('Backup inflations')
ax[0,0].grid(True), ax[0,1].grid(True)

# Figure 4C-D

pressure_high = breaths_all_1['pressure'].max() * 1.1
vol_high = breaths_all_2['volume'].max() * 0.9
    
ax[1,0].scatter(synchronized_all_2['pressure'], synchronized_all_2['volume'], color = 'black', s = 5, alpha = 0.01 )
ax[1,1].scatter(backup_all_2['pressure'], backup_all_2['volume'], color = 'black', s = 5, alpha = 0.01 )
ax[1,0].set_xlim(0, 30), ax[1,1].set_xlim(0, 30)
ax[1,0].set_ylim(0, 25), ax[1,1].set_ylim(0, 25)
ax[1,0].set_xlabel('Pressure (mbar)'), ax[1,1].set_xlabel('Pressure (mbar)')
ax[1,0].set_ylabel('Volume (mL)'), ax[1,1].set_ylabel('Volume (mL)')
ax[1,0].set_title('Synchronised inflations'), ax[1,1].set_title('Backup inflations')
ax[1,0].grid(True), ax[1,1].grid(True)

fig.text(0.04, 0.92, 'A', fontsize = 16) 
fig.text(0.49, 0.92, 'B', fontsize = 16)
fig.text(0.04, 0.48, 'C', fontsize = 16)
fig.text(0.49, 0.48, 'D', fontsize = 16)

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Figure_4', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);

##### Supplementary Figure 4

In [ ]:
dpi = 300
filetype = 'pdf'
breath = breaths_1[2841]

fig = plt.figure(figsize=(12, 18))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace = 0.4)

ax0 = plt.subplot2grid((5, 2), (0, 0), rowspan = 1, colspan=2)
ax1 = plt.subplot2grid((5, 2), (1, 0), rowspan = 1, colspan=2)
ax2 = plt.subplot2grid((5, 2), (2, 0), rowspan = 1, colspan=2)
ax3 = plt.subplot2grid((5, 2), (3, 0), rowspan=2, colspan = 1)
ax4 = plt.subplot2grid((5, 2), (3, 1), rowspan=2, colspan = 1)

breath.pressure.plot(ax = ax0, color = 'red', title = 'Pressure', linewidth=2, 
            ylim = [0, (breath.pressure.max() * 1.2)], x_compat = True);
breath.flow.plot(ax = ax1, color = 'green', title = 'Flow', linewidth=2,
            ylim = [(breath.flow.min() * 1.2), (breath.flow.max() * 1.2)], x_compat = True)
xmin, xmax = ax1.get_xlim()
ax1.hlines(0, xmin, xmax, color = 'black', linewidth = 2)
breath.volume.plot(ax = ax2, color = 'blue', title = 'Volume', linewidth=2, 
            ylim = [-0.1, (breath.volume.max() * 1.2)], x_compat=True)

ax0.get_xaxis().set_ticks([])
ax1.get_xaxis().set_ticks([])

majorFmt = dates.DateFormatter('%H:%M:%S%.%f')  
ax2.xaxis.set_major_formatter(majorFmt)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, 
             horizontalalignment = 'center')
    
ax0.set_xlabel(''); ax1.set_xlabel('')
ax2.set_xlabel('', size = 16, color = 'black', rotation = 0 )
ax0.set_ylabel('mbar', size = 16, color = 'black')
ax1.set_ylabel('L/min', size = 16, color = 'black')
ax2.set_ylabel('mL', size = 16, color = 'black')
ax0.set_title('Pressure', size = 16, color = 'black')
ax1.set_title('Flow', size = 16, color = 'black')
ax2.set_title('Volume', size = 16, color = 'black')

ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray') 
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax2.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
x = breath.pressure
y = breath.flow
z = breath.volume

ax3.plot(x, z, linewidth = 2, color = 'red',)
ax3.set(xlim = [0, (x.max() * 1.2)], ylim = [0, (z.max() * 1.2)])
ax3.arrow(1.5, 2, 2.5, 0, head_width = 0.5, color = 'black')

ax3.set_title('', size = 16, color = 'black')
ax3.set_xlabel('Pressure (mbar)', size = 16, color = 'black')
ax3.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax3.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
ax4.plot(y, z, linewidth = 2, color = 'blue',)
ax4.set(xlim = [y.min() * 1.2, (y.max() * 1.2)], ylim = [0, (z.max() * 1.2)])
ax4.arrow(-2, 2, 1.5, 0, head_width = 0.5, color = 'black')

ax4.set_title('', size = 16, color = 'black')
ax4.set_xlabel('Flow (L/min)', size = 16, color = 'black')
ax4.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax4.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.04, 0.92, 'A', fontsize = 20) 
fig.text(0.04, 0.44, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Supplementary_Figure_4_col', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'pdf'
breath = breaths_1[2841]

fig = plt.figure(figsize=(12, 18))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace = 0.4)

ax0 = plt.subplot2grid((5, 2), (0, 0), rowspan = 1, colspan=2)
ax1 = plt.subplot2grid((5, 2), (1, 0), rowspan = 1, colspan=2)
ax2 = plt.subplot2grid((5, 2), (2, 0), rowspan = 1, colspan=2)
ax3 = plt.subplot2grid((5, 2), (3, 0), rowspan=2, colspan = 1)
ax4 = plt.subplot2grid((5, 2), (3, 1), rowspan=2, colspan = 1)

breath.pressure.plot(ax = ax0, color = 'black', title = 'Pressure', linewidth=2, 
            ylim = [0, (breath.pressure.max() * 1.2)], x_compat = True);
breath.flow.plot(ax = ax1, color = 'black', title = 'Flow', linewidth=2,
            ylim = [(breath.flow.min() * 1.2), (breath.flow.max() * 1.2)], x_compat = True)
xmin, xmax = ax1.get_xlim()
ax1.hlines(0, xmin, xmax, color = 'black', linewidth = 2)
breath.volume.plot(ax = ax2, color = 'black', title = 'Volume', linewidth=2, 
            ylim = [-0.1, (breath.volume.max() * 1.2)], x_compat=True)

ax0.get_xaxis().set_ticks([])
ax1.get_xaxis().set_ticks([])

majorFmt = dates.DateFormatter('%H:%M:%S%.%f')  
ax2.xaxis.set_major_formatter(majorFmt)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, 
             horizontalalignment = 'center')
    
ax0.set_xlabel(''); ax1.set_xlabel('')
ax2.set_xlabel('', size = 16, color = 'black', rotation = 0 )
ax0.set_ylabel('mbar', size = 16, color = 'black')
ax1.set_ylabel('L/min', size = 16, color = 'black')
ax2.set_ylabel('mL', size = 16, color = 'black')
ax0.set_title('Pressure', size = 16, color = 'black')
ax1.set_title('Flow', size = 16, color = 'black')
ax2.set_title('Volume', size = 16, color = 'black')

ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray') 
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax2.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
x = breath.pressure
y = breath.flow
z = breath.volume

ax3.plot(x, z, linewidth = 2, color = 'black',)
ax3.set(xlim = [0, (x.max() * 1.2)], ylim = [0, (z.max() * 1.2)])
ax3.arrow(1.5, 2, 2.5, 0, head_width = 0.5, color = 'black')

ax3.set_title('', size = 16, color = 'black')
ax3.set_xlabel('Pressure (mbar)', size = 16, color = 'black')
ax3.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax3.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
ax4.plot(y, z, linewidth = 2, color = 'black',)
ax4.set(xlim = [y.min() * 1.2, (y.max() * 1.2)], ylim = [0, (z.max() * 1.2)])
ax4.arrow(-2, 2, 1.5, 0, head_width = 0.5, color = 'black')
    
ax4.set_title('', size = 16, color = 'black')
ax4.set_xlabel('Flow (L/min)', size = 16, color = 'black')
ax4.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax4.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.04, 0.92, 'A', fontsize = 20) 
fig.text(0.04, 0.44, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Supplementary_Figure_4', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);

##### Supplementary Figure 5

In [ ]:
dpi = 300
filetype = 'pdf'
breath = breaths_2[73269]

fig = plt.figure(figsize=(12, 18))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace = 0.4)

ax0 = plt.subplot2grid((5, 2), (0, 0), rowspan = 1, colspan=2)
ax1 = plt.subplot2grid((5, 2), (1, 0), rowspan = 1, colspan=2)
ax2 = plt.subplot2grid((5, 2), (2, 0), rowspan = 1, colspan=2)
ax3 = plt.subplot2grid((5, 2), (3, 0), rowspan=2, colspan = 1)
ax4 = plt.subplot2grid((5, 2), (3, 1), rowspan=2, colspan = 1)

breath.pressure.plot(ax = ax0, color = 'red', title = 'Pressure', linewidth=2, 
            ylim = [0, (breath.pressure.max() * 1.2)], x_compat = True);
breath.flow.plot(ax = ax1, color = 'green', title = 'Flow', linewidth=2,
            ylim = [(breath.flow.min() * 1.2), (breath.flow.max() * 1.2)], x_compat = True)
xmin, xmax = ax1.get_xlim()
ax1.hlines(0, xmin, xmax, color = 'black', linewidth = 2)
breath.volume.plot(ax = ax2, color = 'blue', title = 'Volume', linewidth=2, 
            ylim = [-0.1, (breath.volume.max() * 1.2)], x_compat=True)

ax0.get_xaxis().set_ticks([])
ax1.get_xaxis().set_ticks([])

majorFmt = dates.DateFormatter('%H:%M:%S%.%f')  
ax2.xaxis.set_major_formatter(majorFmt)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, 
             horizontalalignment = 'center')
    
ax0.set_xlabel(''); ax1.set_xlabel('')
ax2.set_xlabel('', size = 16, color = 'black', rotation = 0 )
ax0.set_ylabel('mbar', size = 16, color = 'black')
ax1.set_ylabel('L/min', size = 16, color = 'black')
ax2.set_ylabel('mL', size = 16, color = 'black')
ax0.set_title('Pressure', size = 16, color = 'black')
ax1.set_title('Flow', size = 16, color = 'black')
ax2.set_title('Volume', size = 16, color = 'black')

ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray') 
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax2.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
x = breath.pressure
y = breath.flow
z = breath.volume

ax3.plot(x, z, linewidth = 2, color = 'red',)
ax3.set(xlim = [0, (x.max() * 1.2)], ylim = [0, (z.max() * 1.2)])

ax3.set_title('', size = 16, color = 'black')
ax3.set_xlabel('Pressure (mbar)', size = 16, color = 'black')
ax3.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax3.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
ax4.plot(y, z, linewidth = 2, color = 'blue',)
ax4.set(xlim = [y.min() * 1.2, (y.max() * 1.2)], ylim = [0, (z.max() * 1.2)])
    
ax4.set_title('', size = 16, color = 'black')
ax4.set_xlabel('Flow (L/min)', size = 16, color = 'black')
ax4.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax4.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.04, 0.92, 'A', fontsize = 20) 
fig.text(0.04, 0.44, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Supplementary_Figure_5_col', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'pdf'
breath = breaths_2[73269]

fig = plt.figure(figsize=(12, 18))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace = 0.4)

ax0 = plt.subplot2grid((5, 2), (0, 0), rowspan = 1, colspan=2)
ax1 = plt.subplot2grid((5, 2), (1, 0), rowspan = 1, colspan=2)
ax2 = plt.subplot2grid((5, 2), (2, 0), rowspan = 1, colspan=2)
ax3 = plt.subplot2grid((5, 2), (3, 0), rowspan=2, colspan = 1)
ax4 = plt.subplot2grid((5, 2), (3, 1), rowspan=2, colspan = 1)

breath.pressure.plot(ax = ax0, color = 'black', title = 'Pressure', linewidth=2, 
            ylim = [0, (breath.pressure.max() * 1.2)], x_compat = True);
breath.flow.plot(ax = ax1, color = 'black', title = 'Flow', linewidth=2,
            ylim = [(breath.flow.min() * 1.2), (breath.flow.max() * 1.2)], x_compat = True)
xmin, xmax = ax1.get_xlim()
ax1.hlines(0, xmin, xmax, color = 'black', linewidth = 2)
breath.volume.plot(ax = ax2, color = 'black', title = 'Volume', linewidth=2, 
            ylim = [-0.1, (breath.volume.max() * 1.2)], x_compat=True)

ax0.get_xaxis().set_ticks([])
ax1.get_xaxis().set_ticks([])

majorFmt = dates.DateFormatter('%H:%M:%S%.%f')  
ax2.xaxis.set_major_formatter(majorFmt)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, 
             horizontalalignment = 'center')
    
ax0.set_xlabel(''); ax1.set_xlabel('')
ax2.set_xlabel('', size = 16, color = 'black', rotation = 0 )
ax0.set_ylabel('mbar', size = 16, color = 'black')
ax1.set_ylabel('L/min', size = 16, color = 'black')
ax2.set_ylabel('mL', size = 16, color = 'black')
ax0.set_title('Pressure', size = 16, color = 'black')
ax1.set_title('Flow', size = 16, color = 'black')
ax2.set_title('Volume', size = 16, color = 'black')

ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray') 
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax2.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
x = breath.pressure
y = breath.flow
z = breath.volume

ax3.plot(x, z, linewidth = 2, color = 'black',)
ax3.set(xlim = [0, (x.max() * 1.2)], ylim = [0, (z.max() * 1.2)])

ax3.set_title('', size = 16, color = 'black')
ax3.set_xlabel('Pressure (mbar)', size = 16, color = 'black')
ax3.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax3.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
  
ax4.plot(y, z, linewidth = 2, color = 'black',)
ax4.set(xlim = [y.min() * 1.2, (y.max() * 1.2)], ylim = [0, (z.max() * 1.2)])
    
ax4.set_title('', size = 16, color = 'black')
ax4.set_xlabel('Flow (L/min)', size = 16, color = 'black')
ax4.set_ylabel('Volume (mL)', size = 16, color = 'black')
ax4.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.04, 0.92, 'A', fontsize = 20) 
fig.text(0.04, 0.44, 'B', fontsize = 20)

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Supplementary_Figure_5', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);